In [1]:
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [2]:
data = pd.read_csv("data.csv")

In [3]:
MAP = {
    "r1":["r2","r3"],
    "r2":["r1","r4"],
    "r3":["r1","r7"],
    "r4":["r2","r8"],
    "r5":["r6","r9","c3"],
    "r6":["r5","c3"],
    "r7":["r3","c1"],
    "r8":["r4","r9"],
    "r9":["r5","r8","r13"],
    "r10":["c3"],
    "r11":["c3"],
    "r12":["outside","r22"],
    "r13":["r9","r24"],
    "r14":["r24"],
    "r15":["c3"],
    "r16":["c3"],
    "r17":["c3"],
    "r18":["c3"],
    "r19":["c3"],
    "r20":["c3"],
    "r21":["c3"],
    "r22":["r12","r25"],
    "r23":["r24"],
    "r24":["r13","r14","r23"],
    "r25":["r22","r26","c1"],
    "r26":["r25","r27"],
    "r27":["r26","r32"],
    "r28":["c4"],
    "r29":["c4","r30"],
    "r30":["r29"],
    "r31":["r32"],
    "r32":["r27","r31","r33"],
    "r33":["r32"],
    "r34":["c2"],
    "r35":["c4"],
    "c1":["r7","r25","c2"],
    "c2":["c1","r34","c4"],
    "c3": ["r5","r6","r10","r11","r15","r16","r17","r18","r19","r20","r21","o1"],
    "c4":["r29","c2","r35","r28","o1"],
    "o1":["c3","c4"],
    "outside":["r12"]  
}
censor_DF =  {
    "r1": "us3", #us for unreliable_sensor
    "r5": "rs2" ,#rs for reliable_sensor
    "r8": "ds1", #ds for door sensor
    "r9": "ds1",
    "r16": "rs1",
    "r24" : "us4",
    "r25" : "rs3",
    "r26" : "ds3", 
    "r27" : "ds3",
    "r31" : "rs4",
    "r35" : "ds4",
    "c1" : "ds2",
    "c2" : "ds2",
    "c3" : "us2",
    "c4" : "ds4",
    "o1" : "us1"
}

In [4]:
index1 = data[data.time == "08:01:00"].index.tolist()[0] #  get in entrance ends
index2 = data[data.time == "08:05:00"].index.tolist()[0] # People tends to be stable
index3 = data[data.time == "10:00:00"].index.tolist()[0]
index4 = data[data.time == "12:00:00"].index.tolist()[0]
index5 = data[data.time == "14:00:00"].index.tolist()[0]
index6 = data[data.time == "16:00:00"].index.tolist()[0]
index7 = data[data.time=="17:30:00"].index.tolist()[0] # leave work starts
index8 = data[data.time=="17:41:00"].index.tolist()[0] # leave work ends

In [5]:
data1 = data.iloc[0:index1] # 08:00 - 08:01
data2 = data.iloc[index1:index2].reset_index(drop=True) #08:01 - 08:05
data3 = data.iloc[index2:index3].reset_index(drop=True) # 08:05 - 17:30
data4 = data.iloc[index3:index4].reset_index(drop=True) # 17:30 - 17:41
data5 = data.iloc[index4:index5].reset_index(drop=True)
data6 = data.iloc[index5:index6].reset_index(drop=True)
data7 = data.iloc[index6:index7].reset_index(drop=True)
data8 = data.iloc[index7:index8].reset_index(drop=True)
data9 = data.iloc[index8:].reset_index(drop=True)
data_lst = [data1,data2,data3,data4,data5,data6,data7,data8,data9]

In [6]:
# self_trans means the probability of a person keeps staying in the same area
self_trans = {key:[] for key in MAP.keys()} 

for sub_data in data_lst:
    for place in MAP.keys():
        col_data = sub_data[place] 
        stay = 0
        go = 0
        for i in range(1,col_data.shape[0]):
            if(col_data[i]>=col_data[i-1]):
                stay += col_data[i-1]
            else:
                stay += col_data[i]
                go += col_data[i-1] - col_data[i]
        total = stay+go
        if(total!=0):
            self_trans[place].append(stay/total)
        else:
            self_trans[place].append(0)
                    
            
print(self_trans)

{'r1': [0, 1.0, 0.9478584729981379, 0.9451827242524917, 0.891156462585034, 0.8903654485049833, 0.9216300940438872, 0.3333333333333333, 0], 'r2': [0, 0, 0.6363636363636364, 0.6829268292682927, 0.7572815533980582, 0.6388888888888888, 0.0, 0.0, 0], 'r3': [0, 0.9, 0.8888888888888888, 0.8201058201058201, 0.8505747126436781, 0.9572564612326043, 0.9216216216216216, 0.3333333333333333, 0], 'r4': [0, 0, 0.7450980392156863, 0.8571428571428571, 0.8333333333333334, 0.8089171974522293, 0.8235294117647058, 0.5, 0], 'r5': [0, 0, 0.48333333333333334, 0.6164383561643836, 0.7786885245901639, 0.3695652173913043, 0.5283018867924528, 0.0, 0], 'r6': [0, 0, 0, 0.9587628865979382, 0.85, 0.96, 0.9722222222222222, 0, 0], 'r7': [0, 0.0, 0.955617198335645, 0.941656942823804, 0.943646408839779, 0.9330143540669856, 0.9495677233429395, 0.5, 0], 'r8': [0, 0, 0.043478260869565216, 0.0, 0.043478260869565216, 0.4897959183673469, 0.4146341463414634, 0, 0], 'r9': [0, 0, 0.7171717171717171, 0.8493150684931506, 0.7, 0.86411

## Find Neighbour Transition

In [7]:
def n_step_neighbour(node,n,G):
    neighbour_list = []
    neighbour_list = neighbour_list + G[node]
    while n > 1:
        for new_node in neighbour_list:
            neighbour_list = neighbour_list+ G[new_node]
        n = n -1 
    n_list = list(set(neighbour_list))
    n_list.remove(node)
    return(n_list)

In [8]:
neighbour_MAP = {key:n_step_neighbour(key,2,MAP) for key in MAP.keys()}

In [9]:
neighbour_MAP

{'r1': ['r3', 'r4', 'r2', 'r7'],
 'r2': ['r4', 'r3', 'r1', 'r8'],
 'r3': ['c1', 'r1', 'r2', 'r7'],
 'r4': ['r8', 'r1', 'r2', 'r9'],
 'r5': ['r20',
  'r13',
  'r19',
  'r17',
  'r18',
  'o1',
  'r8',
  'r15',
  'r10',
  'r16',
  'r9',
  'r11',
  'r6',
  'r21',
  'c3'],
 'r6': ['r20',
  'r19',
  'r17',
  'r18',
  'o1',
  'r15',
  'r5',
  'r10',
  'r16',
  'r9',
  'r11',
  'r21',
  'c3'],
 'r7': ['r3', 'c1', 'c2', 'r1', 'r25'],
 'r8': ['r13', 'r4', 'r5', 'r2', 'r9'],
 'r9': ['r13', 'r4', 'r8', 'r5', 'r6', 'r24', 'c3'],
 'r10': ['r20',
  'r19',
  'r17',
  'r18',
  'o1',
  'r15',
  'r5',
  'r16',
  'r11',
  'r6',
  'r21',
  'c3'],
 'r11': ['r20',
  'r19',
  'r17',
  'r18',
  'o1',
  'r15',
  'r10',
  'r5',
  'r16',
  'r6',
  'r21',
  'c3'],
 'r12': ['r25', 'r22', 'outside'],
 'r13': ['r14', 'r8', 'r23', 'r5', 'r9', 'r24'],
 'r14': ['r13', 'r23', 'r24'],
 'r15': ['r20',
  'r19',
  'r17',
  'r18',
  'o1',
  'r10',
  'r5',
  'r16',
  'r11',
  'r6',
  'r21',
  'c3'],
 'r16': ['r20',
  'r19',
  

In [10]:
# neigh_trans means the probability of a person transfers to the neighbourhood
neigh_trans = {key:{} for key in MAP.keys()} 
for key,neighbours in neighbour_MAP.items():
    for neighbour in neighbours:
        neigh_trans[key][neighbour]=[]

In [11]:
neigh_trans

{'r1': {'r3': [], 'r4': [], 'r2': [], 'r7': []},
 'r2': {'r4': [], 'r3': [], 'r1': [], 'r8': []},
 'r3': {'c1': [], 'r1': [], 'r2': [], 'r7': []},
 'r4': {'r8': [], 'r1': [], 'r2': [], 'r9': []},
 'r5': {'r20': [],
  'r13': [],
  'r19': [],
  'r17': [],
  'r18': [],
  'o1': [],
  'r8': [],
  'r15': [],
  'r10': [],
  'r16': [],
  'r9': [],
  'r11': [],
  'r6': [],
  'r21': [],
  'c3': []},
 'r6': {'r20': [],
  'r19': [],
  'r17': [],
  'r18': [],
  'o1': [],
  'r15': [],
  'r5': [],
  'r10': [],
  'r16': [],
  'r9': [],
  'r11': [],
  'r21': [],
  'c3': []},
 'r7': {'r3': [], 'c1': [], 'c2': [], 'r1': [], 'r25': []},
 'r8': {'r13': [], 'r4': [], 'r5': [], 'r2': [], 'r9': []},
 'r9': {'r13': [],
  'r4': [],
  'r8': [],
  'r5': [],
  'r6': [],
  'r24': [],
  'c3': []},
 'r10': {'r20': [],
  'r19': [],
  'r17': [],
  'r18': [],
  'o1': [],
  'r15': [],
  'r5': [],
  'r16': [],
  'r11': [],
  'r6': [],
  'r21': [],
  'c3': []},
 'r11': {'r20': [],
  'r19': [],
  'r17': [],
  'r18': [],
  '

In [12]:
for sub_data in data_lst:
    place_data = sub_data.iloc[:,17:58]
    pl_diff = place_data.diff()
    sub_neigh_trans = {key:{} for key in MAP.keys()}
    for key,neighbours in neighbour_MAP.items():
        for neighbour in neighbours:
            sub_neigh_trans[key][neighbour]= 0 
    for place in MAP.keys():
        for i in range(sub_data.shape[0]):
            if(sub_data.iloc[i][place]<sub_data.iloc[i-1][place]): # someone leaves the place
                    for neighbour in neighbour_MAP[place]:
                        if pl_diff.iloc[i][neighbour]>0:
                            sub_neigh_trans[place][neighbour]+=pl_diff.iloc[i][neighbour]
                        else:
                            sub_neigh_trans[place][neighbour]+=0
        neigh_total = sum(sub_neigh_trans[place].values())
        if (neigh_total!=0):
             for neighbour in neighbour_MAP[place]:
                sub_neigh_trans[place][neighbour] = sub_neigh_trans[place][neighbour]/neigh_total  
                neigh_trans[place][neighbour].append(sub_neigh_trans[place][neighbour])
        else:
            for neighbour in neighbour_MAP[place]:
                neigh_trans[place][neighbour].append(0) 

print(neigh_trans)

{'r1': {'r3': [0, 0, 0.45454545454545453, 0.3076923076923077, 0.4722222222222222, 0.4864864864864865, 0.35714285714285715, 0.5, 0], 'r4': [0, 0, 0.15151515151515152, 0.20512820512820512, 0.05555555555555555, 0.10810810810810811, 0.0, 0.0, 0], 'r2': [0, 0, 0.2727272727272727, 0.2564102564102564, 0.3611111111111111, 0.32432432432432434, 0.39285714285714285, 0.5, 0], 'r7': [0, 0, 0.12121212121212122, 0.23076923076923078, 0.1111111111111111, 0.08108108108108109, 0.25, 0.0, 0]}, 'r2': {'r4': [0, 0, 0.42857142857142855, 0.4838709677419355, 0.40625, 0.41379310344827586, 0.39285714285714285, 0.0, 0], 'r3': [0, 0, 0.10714285714285714, 0.06451612903225806, 0.03125, 0.034482758620689655, 0.0, 0.0, 0], 'r1': [0, 0, 0.39285714285714285, 0.45161290322580644, 0.5, 0.4482758620689655, 0.4642857142857143, 1.0, 0], 'r8': [0, 0, 0.07142857142857142, 0.0, 0.0625, 0.10344827586206896, 0.14285714285714285, 0.0, 0]}, 'r3': {'c1': [0, 0.0, 0.05128205128205128, 0.02631578947368421, 0.16, 0.21052631578947367, 0

In [13]:
true_neigh_trans = copy.deepcopy(neigh_trans)

In [14]:
for place, neighbours in true_neigh_trans.items():
    place_tran_prob = [1-val for val in self_trans[place]]
    for k,v in neighbours.items():
        for i in range(len(v)):
            true_neigh_trans[place][k][i] = true_neigh_trans[place][k][i] * place_tran_prob[i] 
            
            
print(true_neigh_trans)

{'r1': {'r3': [0, 0.0, 0.023700694091755518, 0.016866854076156394, 0.05139833711262283, 0.053335727754332436, 0.02798925212718315, 0.33333333333333337, 0], 'r4': [0, 0.0, 0.007900231363918507, 0.011244569384104262, 0.006046863189720333, 0.011852383945407207, 0.0, 0.0, 0], 'r2': [0, 0.0, 0.01422041645505331, 0.014055711730130326, 0.039304610733182165, 0.03555715183622162, 0.030788177339901465, 0.33333333333333337, 0], 'r7': [0, 0.0, 0.006320185091134805, 0.012650140557117296, 0.012093726379440665, 0.008889287959055405, 0.019592476489028204, 0.0, 0]}, 'r2': {'r4': [0, 0, 0.15584415584415584, 0.15342250196695514, 0.09860436893203883, 0.14942528735632185, 0.39285714285714285, 0.0, 0], 'r3': [0, 0, 0.03896103896103896, 0.020456333595594015, 0.00758495145631068, 0.012452107279693488, 0.0, 0.0, 0], 'r1': [0, 0, 0.14285714285714285, 0.1431943351691581, 0.12135922330097088, 0.16187739463601536, 0.4642857142857143, 1.0, 0], 'r8': [0, 0, 0.025974025974025972, 0.0, 0.01516990291262136, 0.037356321

## Transition Matrix Generation

In [15]:
for i in range(len(data_lst)):
    tran_lst = [] 
    for place in MAP.keys():
        tran_row = {}
        tran_row['place'] = place 
        for col_place in MAP.keys():
            if(place==col_place):
                tran_row[col_place] = round(self_trans[col_place][i],4)
            else:
                if (col_place in true_neigh_trans[place].keys()):
                    tran_row[col_place] = round(true_neigh_trans[place][col_place][i],2)
                else:
                    tran_row[col_place] = 0 
        tran_lst.append(tran_row)
    tran_matrix = pd.DataFrame(tran_lst)
    file_name = "tran_matrix" + str(i) + ".csv"
    tran_matrix.to_csv(file_name)